In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

In [2]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/modelv1'

In [3]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
word_state_size = 100
other_state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 10

In [4]:
with tf.name_scope("input"):
    path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length")
    word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size], name="y")

In [5]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [6]:
word_hidden_state = tf.zeros([batch_size, word_state_size], name='word_hidden_state')
word_cell_state = tf.zeros([batch_size, word_state_size], name='word_cell_state')
word_init_state = tf.contrib.rnn.LSTMStateTuple(word_hidden_state, word_cell_state)

other_hidden_states = tf.zeros([channels-1, batch_size, other_state_size], name="hidden_state")
other_cell_states = tf.zeros([channels-1, batch_size, other_state_size], name="cell_state")

other_init_states = [tf.contrib.rnn.LSTMStateTuple(other_hidden_states[i], other_cell_states[i]) for i in range(channels-1)]

with tf.variable_scope("word_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=word_init_state)
    state_series_word1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("word_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[1], sequence_length=path_length[1], initial_state=word_init_state)
    state_series_word2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=other_init_states[0])
    state_series_pos1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=other_init_states[0])
    state_series_pos2 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=other_init_states[1])
    state_series_dep1 = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=other_init_states[1])
    state_series_dep2 = tf.reduce_max(state_series, axis=1)

state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

state_series = tf.concat([state_series1, state_series2], 1)

In [7]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer, W) + b
    predictions = tf.argmax(logits, 1)

In [8]:
tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('biases')==-1):
            tv_regu.append(t)

with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

global_step = tf.Variable(0, name="global_step")

optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

In [9]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

In [10]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()

In [11]:
pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

In [15]:

rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [16]:
f = open('data/train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open('data/train_relations.txt'):
    relations.append(line.strip().split()[1])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [18]:
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [19]:
model = tf.train.latest_checkpoint(model_dir)
saver.restore(sess, model)

INFO:tensorflow:Restoring parameters from checkpoint/modelv1/model


In [13]:
# latest_embd = tf.train.latest_checkpoint(word_embd_dir)
# word_embedding_saver.restore(sess, latest_embd)

In [20]:
num_epochs = 10
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        if step%10==0:
            print("Step:", step, "loss:",loss)
        if step % 1000 == 0:
            saver.save(sess, model_dir + '/model')
            print("Saved Model")

In [21]:
# training accuracy
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

training accuracy 99.2625


In [22]:
f = open('data/test_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open('data/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [23]:
# test 
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("test accuracy", accuracy)

test accuracy 61.4022140221
